In [1]:
import matplotlib.pyplot as plt
import itertools
import numpy as np
import pandas as pd
from keras.utils.np_utils import to_categorical
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential, load_model
from keras.datasets import mnist
import itertools
import random

import sys
sys.path.append('../')
from Core_Functions import multi_hot_encoding

Using TensorFlow backend.


In [2]:
sys.path

['/Users/feldnerd/Documents/GitHub/MNIST_FindingMino/Question_2_can_a_NN_learn_the_ordinal_relationship_of_MNIST_digits',
 '/usr/local/Cellar/python@3.9/3.9.2_4/Frameworks/Python.framework/Versions/3.9/lib/python39.zip',
 '/usr/local/Cellar/python@3.9/3.9.2_4/Frameworks/Python.framework/Versions/3.9/lib/python3.9',
 '/usr/local/Cellar/python@3.9/3.9.2_4/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload',
 '',
 '/usr/local/lib/python3.9/site-packages',
 '/usr/local/Cellar/protobuf/3.13.0_1/libexec/lib/python3.9/site-packages',
 '/usr/local/Cellar/sip/6.0.3_1/libexec/lib/python3.9/site-packages',
 '/usr/local/lib/python3.9/site-packages/IPython/extensions',
 '/Users/feldnerd/.ipython',
 '../']

In [3]:
# Randomly select a defined number of image paires from a given dataset
def get_left_min_image_combinations(number_image_pairs_selected, image_data, image_labels):
    
    num_images_total = len(image_labels)
    
    image_combination_data = []
    image_combination_labels = []
    
    for i in range(number_image_pairs_selected):
        
        # Draw two image indices from a uniform random distribution
        random_index_A = random.randint(0, num_images_total-1)
        random_index_B = random.randint(0, num_images_total-1)
        
        # Randomly choose two images from the dataset
        image_A = image_data[random_index_A]
        image_B = image_data[random_index_B]
        
        # Merge images TODO: this could be a function with several options
        combined_images = np.hstack((image_A, image_B))
        
        # Find the minimum between the two labels
        label_A = image_labels[random_index_A]
        label_B = image_labels[random_index_B]
        
        if label_A < label_B:
            combined_images = np.hstack((image_A, image_B))
            image_combination_labels.append([label_A, label_B])
            # Append newly generated image combination and minimum label to list
            image_combination_data.append(combined_images)
            
        elif label_A > label_B: 
            # Merge images TODO: this could be a function with several options
            combined_images = np.hstack((image_B, image_A))
            image_combination_labels.append([label_A, label_B])
            # Append newly generated image combination and minimum label to list
            image_combination_data.append(combined_images)
        
    # Convert image data and labels lists to numpy arrays
    image_combination_data = np.array(image_combination_data)
    image_combination_labels = np.array(image_combination_labels)
    
    return image_combination_data, image_combination_labels

In [4]:
# Randomly select a defined number of image paires from a given dataset
def get_right_min_image_combinations(number_image_pairs_selected, image_data, image_labels):
    
    num_images_total = len(image_labels)
    
    image_combination_data = []
    image_combination_labels = []
    
    for i in range(number_image_pairs_selected):
        
        # Draw two image indices from a uniform random distribution
        random_index_A = random.randint(0, num_images_total-1)
        random_index_B = random.randint(0, num_images_total-1)
        
        # Randomly choose two images from the dataset
        image_A = image_data[random_index_A]
        image_B = image_data[random_index_B]
        
        # Merge images TODO: this could be a function with several options
        combined_images = np.hstack((image_A, image_B))
        
        # Find the minimum between the two labels
        label_A = image_labels[random_index_A]
        label_B = image_labels[random_index_B]
        #minimum_label = min(label_A, label_B)
        
        if label_A < label_B:

            combined_images = np.hstack((image_B, image_A))
            image_combination_labels.append([label_A, label_B])
            # Append newly generated image combination and minimum label to list
            image_combination_data.append(combined_images)
            
        elif label_A > label_B: 

            combined_images = np.hstack((image_A, image_B))
            image_combination_labels.append([label_A, label_B])
            # Append newly generated image combination and minimum label to list
            image_combination_data.append(combined_images)
        # Here we want to exclude the images where they are the same on both sides
            
   

        
        
    # Convert image data and labels lists to numpy arrays
    image_combination_data = np.array(image_combination_data)
    image_combination_labels = np.array(image_combination_labels)
    
    return image_combination_data, image_combination_labels

In [5]:
# Load data and split into training and test
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [6]:
# Split training set further into training and validation sets
sample_size = 40000
X_train, X_val, y_train, y_val = train_images[:sample_size], train_images[sample_size:sample_size*2],  train_labels[:sample_size], train_labels[sample_size:sample_size*2]

In [7]:
combo_train_X, combo_train_y = get_left_min_image_combinations(40000, X_train, y_train)
combo_val_X, combo_val_y = get_right_min_image_combinations(20000, X_val, y_val)

In [8]:
combo_train_X.shape[0]

36041

In [9]:
combo_val_X.shape

(17883, 28, 56)

In [10]:
# Reshape the data so they can be fed into the network, one-hot-encode the labels

combo_train_X = combo_train_X.reshape(combo_train_X.shape[0], 784).astype("float32") / 255

combo_val_X = combo_val_X.reshape(combo_val_X.shape[0], 784).astype("float32") / 255


# train_labels = to_categorical(combo_train_y)
# val_labels = to_categorical(combo_val_y)

ValueError: cannot reshape array of size 56512288 into shape (36041,784)

In [91]:
combo_train_y.shape

(35934, 2)

In [92]:
train_labels[:10]

array([5, 0, 4, 1, 9, 2, 1, 3, 1, 4], dtype=uint8)

In [93]:
# Here instead of using to_categorical, I use my custom multi-hot encoding
"""
train_labels = to_categorical(combo_train_y)
val_labels = to_categorical(combo_val_y)
"""

train_labels = np.array(multi_hot_encoding.multi_hot_encode(combo_train_y))
val_labels =  np.array(multi_hot_encoding.multi_hot_encode(combo_val_y))

In [94]:
combo_train_X.shape

(35934, 28, 56)

In [95]:
train_labels.shape

(35934, 10)

In [96]:
val_labels.shape

(18070, 10)

In [97]:
# Experiment parameters

num_epochs = 5
num_iterations = 5

In [102]:
# Model architecture
def build_CNN():

    # Convolutional NN
    model = tf.keras.Sequential()
    model.add(layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (28,56,1)))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    model.add(layers.MaxPooling2D(2,2))
    model.add(layers.Conv2D(64, (3,3), activation = 'relu'))
    # Adding a NN Classifier
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation = 'relu'))
    model.add(layers.Dense(10, activation = 'sigmoid'))

    model.compile(optimizer='rmsprop', 
                loss='categorical_crossentropy', 
                metrics=['accuracy', 'mae'])

    return model

In [103]:
CNN_model = build_CNN()
CNN_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 26, 54, 32)        320       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 13, 27, 32)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 11, 25, 64)        18496     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 5, 12, 64)         0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 3, 10, 64)         36928     
_________________________________________________________________
flatten_4 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_8 (Dense)              (None, 64)               

In [ ]:
# Fit model
CNN_accuracy = []


CNN_model.fit(train_images, 
          train_labels, 
          epochs = num_epochs, 
          batch_size = 1)
    
# Print loss, mae and accuracy
CNN_eval_model = CNN_model.evaluate(val_images, val_labels)
CNN_accuracy.append(CNN_eval_model[1])

In [109]:
def build_DNN_10_outputs():

    inputs = keras.Input(shape=(28*28*28,))

    dense = layers.Dense(512, activation="relu")
    x = dense(inputs)

    outputs = layers.Dense(10, "sigmoid")(x)
    
    model = keras.Model(inputs = inputs,
                       outputs = outputs)

    model.compile(
        loss=keras.losses.BinaryCrossentropy(),
        optimizer=keras.optimizers.RMSprop(),
        metrics=["accuracy", "mae"],
    )
    
    return model

In [110]:
dnn_model = build_DNN_10_outputs()

In [111]:
dnn_model.fit(train_images, 
          train_labels, 
          epochs = 20, 
          batch_size = 1)

Epoch 1/20


ValueError: in user code:

    /usr/local/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:855 train_function  *
        return step_function(self, iterator)
    /usr/local/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:845 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /usr/local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /usr/local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /usr/local/lib/python3.9/site-packages/tensorflow/python/distribute/distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kwargs)
    /usr/local/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:838 run_step  **
        outputs = model.t